In [ ]:
import pip
#pip.main(['install','systemml==1.1.0'])
#pip.main(['uninstall','systemml==1.1.0'])

!pip install --upgrade https://github.com/niketanpansare/future_of_data/raw/master/systemml-1.1.0-SNAPSHOT-python.tar.gz
!ln -s -f ~/local/lib/python3.5/site-packages/systemml/systemml-java/*.jar ~/data/libs

In [ ]:
!pip show systemml

In [ ]:
#Notes

#from systemml import MLContext, dmlFromFile, dml

#ml = MLContext(sc)
#print(ml.info())
#X = ...//RDD etc.
#script = dmlFromFile("LinearRegCG.dml").input("X", X).input(...).output("B")
#script = dml("""print('Hello World!!')""")
#ml.execute(script)
#b = ml.execute(script).getDataFrame("B")

In [ ]:
from systemml import MLContext, dmlFromFile, dml
ml = MLContext(sc)
script = dml("""s = 'Hello World!!'""").output("s")
str = ml.execute(script).get("s")
print(str)

In [ ]:
#Notes

#from systemml import LinearRegression

#train_df = #...RDD etc
#regr = LinearRegression(spark)
#regr.fit(train_df)
#b = regr.predict(test_df)


In [ ]:
#Notes

#from systemml import Keras2DML

#X_train, Y_train = ...
#sysml_model = Keras2DML(spark, keras_model,...)
#sysml_model.fit(X_train, Y_train)
#b = sysml_model.predict(X_test, Y_test)

In [ ]:
sc.version

In [ ]:
from systemml import MLContext, dmlFromFile, dml
import sys, os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
plt.switch_backend("agg")


In [ ]:
script = """ X = rand(rows = $nr, cols = 1000, sparsity = 0.5)
A = t(X) %*% X
s = sum(A)"""
prog = dml(script).input("$nr", 1e6).output("s")
s = ml.execute(prog).get("s")
print(s)

In [ ]:
%matplotlib inline

In [ ]:
diabetes = datasets.load_diabetes()
diabetes_X = diabetes.data[:, np.newaxis, 2]
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]
diabetes_Y_train = np.matrix(diabetes.target[:-20]).T
diabetes_Y_test = np.matrix(diabetes.target[-20:]).T

print(diabetes_X_train.shape)
print(diabetes_X_test.shape)
print(diabetes_Y_train.shape)
print(diabetes_Y_test.shape)

#plt.scatter(diabetes.data, diabetes.target, color='blue')
plt.scatter(diabetes_X_train, diabetes_Y_train, color='black')
plt.scatter(diabetes_X_test, diabetes_Y_test, color='red')

print(diabetes.data.shape)
print(diabetes.target.shape)
print(diabetes_X.shape)

In [ ]:
#Linear Regression

script = """
    # Add constant feature to X to model intercept
    ones = matrix(1, rows=nrow(X), cols=1)
    X = cbind(X, ones)
    A = t(X) %*% X
    b = t(X) %*% y
    w = solve(A,b)
    bias = as.scalar(w[nrow(w),1])
    w = w[1:nrow(w) - 1,]
    """

prog = dml(script).input(X=diabetes_X_train, y=diabetes_Y_train).output('w', 'bias')
w, bias = ml.execute(prog).get('w', 'bias')
w = w.toNumPy()

In [ ]:
plt.scatter(diabetes_X_train, diabetes_Y_train, color='black')
plt.scatter(diabetes_X_test, diabetes_Y_test, color='red')

plt.plot(diabetes_X_test, (w * diabetes_X_test) * bias, color='blue', linestyle='dotted')

In [ ]:
from pyspark.sql import SQLContext
from systemml import mllearn
#from systemml.mllearn import LinearRegression

sql=SQLContext(sc)
regr=mllearn.LinearRegression(sql)
regr.fit(diabetes_X_train, diabetes_Y_train)
predictions=regr.predict(diabetes_X_test)